In [177]:
import numpy as np
import sympy as sym
from sympy import init_session
from sympy import *
from IPython.display import display
init_printing(use_latex='mathjax')
from matplotlib import pyplot as plt

In [178]:
#A=sym.Matrix([[0,1,1,1,0],[0,0,h,2*h,1],[1/h,0,(h)/2,4*(h)/2,-1],[0,0,(h**3)/6,8*(h**3)/6,(h**2)/2],[0,0,(h**4)/24,16*(h**4)/24,-(h**3)/6]])                                        
h = sym.symbols('h')
y1,y2=sym.symbols('y1 y2')
thetaa,thetab=sym.symbols('theta1 theta2')
row1=[1,1,y1]
row2=[thetaa,thetab,y2]

A=sym.Matrix([row1,row2])


In [179]:
c1,c2=sym.symbols('c_{1} c_{2}')
solve_linear_system(A, c1,c2)

⎧       -θ₂⋅y₁ + y₂         θ₁⋅y₁ - y₂⎫
⎨c_{1}: ───────────, c_{2}: ──────────⎬
⎩         θ₁ - θ₂            θ₁ - θ₂  ⎭

In [180]:
 x = sym.symbols('x')
NL=2*(-0.5+x)*(-1+x)
NC=1-4*(-0.5+x)**2
NR=2*(x-0)*(-0.5+x)


In [181]:
def A(x,xk,xk1):
    k= (x-xk)/(xk1-xk)
    y=1
    return 1

def G(x,xk,xk1):
    k=(x-xk)/(xk1-xk)
    y=1
    return y
def H(x,xk,xk1):
    k=(x-xk)/(xk1-xk)
    y=1
    return y

    
def integrand(Ni,Nj,xreal,b):
    #A=sym.symbols('A')
    #G=sym.symbols('G')
    #hi,hf = sym.symbols('h_{i} h_{f}')
    hi,hf = 0,1
    DNi=diff(Ni, x)
    DNj=diff(Nj, x)
    xk=xreal
    xk1=xreal+b
    part1=integrate(A(x,xk,xk1)*DNi*DNj, (x, hi, hf))
    part2=integrate(G(x,xk,xk1)*Ni*Nj, (x, hi, hf))
    complete=part1+part2
    return complete
    
def integrand2(Ni,Nj,xreal,b,start,end):
    #A=sym.symbols('A')
    #G=sym.symbols('G')
    #hi,hf = sym.symbols('h_{i} h_{f}')
    hi,hf=0,1
    Q=1
    #hi,hf=0,1
    DNi=diff(Ni, x)
    DNj=diff(Nj, x)
    xk=xreal
    xk1=xreal+b
    u=1
    if(start):
        return 0
    elif(end):
        part1=integrate(H(x,xk,xk1)*Ni, (x, hi, hf))-Q
        return part1
    else:
        part1=integrate(H(x,xk,xk1)*Ni, (x, hi, hf))
        part2=Nj.subs(x,0)
        return  part1-part2
        

In [182]:
 
def createlocal(NL,NC,NR,x,b):
    Ni=[NL,NC,NR]
    Nj=[NL,NC,NR]
    n=len(Ni)
    K=zeros(n,n)
    for i in range(0,n):
        for j in range(0,n):
            K[i,j]=integrand(Ni[i],Nj[j],x,b)
    return K
    

In [183]:
def locagloba(localA,n,size):
    row=len(localA[0,:]);
    col=len(localA[:,0]);
    size=size-row;
    if(n>0):
        newA=localA.row_insert(0, zeros(n,row))
        newA=localA.col_insert(0, zeros(col+n,n))
    newA=localA.row_insert(row, zeros(size-n,row))
    newA=localA.col_insert(col, zeros(col+(size-n),(size-n)))  
    return newA

In [184]:
def locagloba(localA,n,size):
    test=localA
    row=len(localA[0,:]);
    col=len(localA[:,0]);
    row=len(localA[0,:]);
    col=len(localA[:,0]);
    if(n>0):
        test=test.row_insert(0, zeros(2*n,row))
        row,col=test.shape
        test=test.col_insert(0, zeros(row,2*n))
    row,col=test.shape

    test=test.row_insert(row, zeros(size-col,row))
    row,col=test.shape
    test=test.col_insert(col, zeros(row,size-col))
    return test

def createglobal(localA,elementnumber):
    number=(3+2*(elementnumber-1))
    globalA=zeros(number)
    for i in range(0,elementnumber):
        new=locagloba(localA[i],i,number)
        globalA=globalA+new
    return globalA
'''
this didn't work just did it by hand 
def createf(NL,NC,NR,size,domain):
    start=domain[0]
    end=domain[1]
    size=size-1
    b=(end-start)/(size)
    Ni=[NL,NC,NR]
    Nj=[NL,NC,NR]
    rhs=zeros(size+1,1)
    for i in range(0,size+1):
        xreal=start+i*b
        start=False;
        end=False;
        if(i==0):
            start=True
        if(i==size):
            end=True
        rhs[i]=integrand2(Ni[i%3],Ni[0],xreal,b,start,end)
    return rhs
'''

"\nthis didn't work just did it by hand \ndef createf(NL,NC,NR,size,domain):\n    start=domain[0]\n    end=domain[1]\n    size=size-1\n    b=(end-start)/(size)\n    Ni=[NL,NC,NR]\n    Nj=[NL,NC,NR]\n    rhs=zeros(size+1,1)\n    for i in range(0,size+1):\n        xreal=start+i*b\n        start=False;\n        end=False;\n        if(i==0):\n            start=True\n        if(i==size):\n            end=True\n        rhs[i]=integrand2(Ni[i%3],Ni[0],xreal,b,start,end)\n    return rhs\n"

In [185]:
elementnumber=2
localA=[]
domain=[0,0.5]
hsize=(domain[1]-domain[0])/elementnumber  
for i in range(0,elementnumber):  
    localA.append(createlocal(NL,NC,NR,domain[0]+i*hsize,hsize))
globalA=createglobal(localA,elementnumber)
sizerow,sizecol=globalA.shape
                  

In [186]:
display(globalA[1:,1:])

⎡5.86666666666666        -2.6                0                  0        ⎤
⎢                                                                        ⎥
⎢      -2.6        4.93333333333333         -2.6        0.299999999999999⎥
⎢                                                                        ⎥
⎢       0                -2.6         5.86666666666666        -2.6       ⎥
⎢                                                                        ⎥
⎣       0          0.299999999999999        -2.6        2.46666666666667 ⎦

In [194]:

F=Matrix([0.166666666666667,0.666666666666667,0.333333333333333,0.666666666666667,1.16666666666667])


In [195]:
M=globalA.col_insert(sizecol, F)
M=M[1:,1:]
display(M)
d0,d1,d2,d3,d4=sym.symbols('d_{0} d_{1} d_{2} d_{3} d_{4}')
D=[d1,d2,d3,d4]
solve_linear_system(M,*D)


⎡5.86666666666666        -2.6                0                  0          0.6
⎢                                                                             
⎢      -2.6        4.93333333333333         -2.6        0.299999999999999  0.3
⎢                                                                             
⎢       0                -2.6         5.86666666666666        -2.6         0.6
⎢                                                                             
⎣       0          0.299999999999999        -2.6        2.46666666666667   1.1

66666666666667⎤
              ⎥
33333333333333⎥
              ⎥
66666666666667⎥
              ⎥
6666666666667 ⎦

{d_{1}: 0.513302277160571, d_{2}: 0.901810266413595, d_{3}: 1.26544352315806, 
d_{4}: 1.69713922173793}

Find the F vector 

In [189]:
integrate(NL,(x,0,1))

0.166666666666667

In [190]:
integrate(NC,(x,0,1))

0.666666666666667

In [191]:

integrate(NR,(x,0,1))+integrate(NL,(x,0,1))


0.333333333333333

In [192]:
integrate(NC,(x,0,1))

0.666666666666667

In [193]:
1+integrate(NR,(x,0,1))

1.16666666666667